In [1]:
print("hello world")

hello world


In [2]:
from pypdf import PdfReader

reader = PdfReader("data/samsung.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[79]
text = page.extract_text()
obj = page.get_object()

In [3]:
text

'2. 연결재무제표\n2-1. 연결 재무상태표 \n연결 재무상태표\n제 55 기          2023.12.31 현재\n제 54 기          2022.12.31 현재\n제 53 기          2021.12.31 현재\n(단위 : 백만원)\n\u3000 제 55 기 제 54 기 제 53 기\n자산 \u3000 \u3000 \u3000\n\u3000유동자산 195,936,557 218,470,581 218,163,185\n\u3000\u3000현금및현금성자산 (주4,28) 69,080,893 49,680,710 39,031,415\n\u3000\u3000단기금융상품 (주4,28) 22,690,924 65,102,886 81,708,986\n\u3000\u3000단기상각후원가금융자산 (주4,28) 608,281 414,610 3,369,034\n\u3000\u3000단기당기손익-공정가치금융자산 (주4,6,28) 27,112 29,080 40,757\n\u3000\u3000매출채권 (주4,5,7,28) 36,647,393 35,721,563 40,713,415\n\u3000\u3000미수금 (주4,7,28) 6,633,248 6,149,209 4,497,257\n\u3000\u3000선급비용 3,366,130 2,867,823 2,336,252\n\u3000\u3000재고자산 (주8) 51,625,874 52,187,866 41,384,404\n\u3000\u3000기타유동자산 (주4,28) 5,038,838 6,316,834 5,081,665\n\u3000\u3000매각예정분류자산 (주33) 217,864 0 0\n\u3000비유동자산 259,969,423 229,953,926 208,457,973\n\u3000\u3000기타포괄손익-공정가치 측정 비유동금융자산 (주4,6,28) 7,481,297 11,397,012 13,965,839\n\u3000\u3000당기손익-공정가치금융자산 (주4,6,28) 1,431,394 1,405,468 1,525,3

In [4]:
obj

{'/Parent': {'/Type': '/Pages',
  '/Count': 540,
  '/Kids': [IndirectObject(1183, 0, 1904512844944),
   IndirectObject(1274, 0, 1904512844944),
   IndirectObject(1337, 0, 1904512844944),
   IndirectObject(2, 0, 1904512844944),
   IndirectObject(8, 0, 1904512844944),
   IndirectObject(11, 0, 1904512844944),
   IndirectObject(14, 0, 1904512844944),
   IndirectObject(16, 0, 1904512844944),
   IndirectObject(18, 0, 1904512844944),
   IndirectObject(20, 0, 1904512844944),
   IndirectObject(24, 0, 1904512844944),
   IndirectObject(26, 0, 1904512844944),
   IndirectObject(28, 0, 1904512844944),
   IndirectObject(29, 0, 1904512844944),
   IndirectObject(32, 0, 1904512844944),
   IndirectObject(34, 0, 1904512844944),
   IndirectObject(36, 0, 1904512844944),
   IndirectObject(38, 0, 1904512844944),
   IndirectObject(40, 0, 1904512844944),
   IndirectObject(41, 0, 1904512844944),
   IndirectObject(43, 0, 1904512844944),
   IndirectObject(46, 0, 1904512844944),
   IndirectObject(48, 0, 19045128449

In [6]:
import tabula

dfs = tabula.read_pdf(
    "data/samsung.pdf",
	pages="80-83",
	stream=True
)
print(f"Data Type :{type(dfs)}")
print(f"Data Length: {len(dfs)}")
for index, table in enumerate(dfs):
    print(f"\nData Index: {index}")
    print(type(table))
    print(table)

Data Type :<class 'list'>
Data Length: 4

Data Index: 0
<class 'pandas.core.frame.DataFrame'>
                          Unnamed: 0  제 55 기   Unnamed: 1  제 54 기  \
0                                 자산     NaN          NaN     NaN   
1                               유동자산     NaN  195,936,557     NaN   
2                   현금및현금성자산 (주4,28)     NaN   69,080,893     NaN   
3                     단기금융상품 (주4,28)     NaN   22,690,924     NaN   
4                단기상각후원가금융자산 (주4,28)     NaN      608,281     NaN   
5          단기당기손익-공정가치금융자산 (주4,6,28)     NaN       27,112     NaN   
6                   매출채권 (주4,5,7,28)     NaN   36,647,393     NaN   
7                      미수금 (주4,7,28)     NaN    6,633,248     NaN   
8                               선급비용     NaN    3,366,130     NaN   
9                          재고자산 (주8)     NaN   51,625,874     NaN   
10                    기타유동자산 (주4,28)     NaN    5,038,838     NaN   
11                    매각예정분류자산 (주33)     NaN      217,864     NaN   
12       

c:\Users\Owner\anaconda3\envs\pdftorag\lib\site-packages\tabula\io.py:1057: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


In [10]:
dfs[0]

,Unnamed: 0,제 55 기,Unnamed: 1,제 54 기,Unnamed: 2,제 53 기,Unnamed: 3
0,자산,NaN,NaN,NaN,NaN,NaN,NaN
1,유동자산,NaN,"195,936,557",NaN,"218,470,581",NaN,"218,163,185"
2,"현금및현금성자산 (주4,28)",NaN,"69,080,893",NaN,"49,680,710",NaN,"39,031,415"
3,"단기금융상품 (주4,28)",NaN,"22,690,924",NaN,"65,102,886",NaN,"81,708,986"
4,"단기상각후원가금융자산 (주4,28)",NaN,"608,281",NaN,"414,610",NaN,"3,369,034"
5,"단기당기손익-공정가치금융자산 (주4,6,28)",NaN,"27,112",NaN,"29,080",NaN,"40,757"
6,"매출채권 (주4,5,7,28)",NaN,"36,647,393",NaN,"35,721,563",NaN,"40,713,415"
7,"미수금 (주4,7,28)",NaN,"6,633,248",NaN,"6,149,209",NaN,"4,497,257"
8,선급비용,NaN,"3,366,130",NaN,"2,867,823",NaN,"2,336,252"
9,재고자산 (주8),NaN,"51,625,874",NaN,"52,187,866",NaN,"41,384,404"


#### trim돼서 계층을 알 수 없으니, pypdf로 나온 값을 참고해서, 맨 왼쪽꺼는 \n 다음에 \u3000이 몇번 나왔는지로 계층을 판단하자.